In [2]:
import pandas as pd

In [3]:
import numpy as np

class MetricCalculator:
    def __init__(self, k=10):
        """
        Initialize the MetricCalculator with an optional parameter k
        which defines the number of top results considered for ranking-based metrics.
        """
        self.k = k

    def hit_rate(self, expected_ids, actual_ids):
        """
        Hit Rate (HR) measures whether the expected item is in the actual top-k list.
        """
        return int(any([item in actual_ids[:self.k] for item in expected_ids]))

    def mrr(self, expected_ids, actual_ids):
        """
        Mean Reciprocal Rank (MRR) evaluates how soon the relevant item appears in the result list.
        """
        for rank, item in enumerate(actual_ids[:self.k], start=1):
            if item in expected_ids:
                return 1 / rank
        return 0

    def precision(self, expected_ids, actual_ids):
        """
        Precision is the fraction of retrieved documents that are relevant.
        """
        relevant_items = [item for item in actual_ids[:self.k] if item in expected_ids]
        return len(relevant_items) / len(actual_ids[:self.k])

    def recall(self, expected_ids, actual_ids):
        """
        Recall is the fraction of relevant documents that are retrieved.
        """
        relevant_items = [item for item in actual_ids[:self.k] if item in expected_ids]
        return len(relevant_items) / len(expected_ids)

    def average_precision(self, expected_ids, actual_ids):
        """
        Average Precision (AP) measures the precision at the rank of each relevant document.
        """
        relevant_count = 0
        precision_sum = 0
        for i, item in enumerate(actual_ids[:self.k]):
            if item in expected_ids:
                relevant_count += 1
                precision_sum += relevant_count / (i + 1)
        return precision_sum / len(expected_ids) if expected_ids else 0

    def dcg(self, actual_ids, expected_ids):
        """
        Discounted Cumulative Gain (DCG) is a measure of ranking quality.
        """
        dcg = 0
        for i, item in enumerate(actual_ids[:self.k]):
            if item in expected_ids:
                dcg += 1 / np.log2(i + 2)
        return dcg

    def idcg(self, expected_ids):
        """
        Ideal DCG (IDCG) is the best possible DCG score if all relevant documents are ranked at the top.
        """
        idcg = 0
        for i in range(min(len(expected_ids), self.k)):
            idcg += 1 / np.log2(i + 2)
        return idcg

    def ndcg(self, expected_ids, actual_ids):
        """
        Normalized Discounted Cumulative Gain (nDCG) is the ratio of DCG to IDCG.
        """
        dcg_score = self.dcg(actual_ids, expected_ids)
        idcg_score = self.idcg(expected_ids)
        return dcg_score / idcg_score if idcg_score > 0 else 0

    def evaluate(self, expected_ids, actual_ids):
        """
        Evaluate all metrics for the given expected and actual IDs.
        """
        metrics = {
            "hit_rate": self.hit_rate(expected_ids, actual_ids),
            "mrr": self.mrr(expected_ids, actual_ids),
            "precision": self.precision(expected_ids, actual_ids),
            "recall": self.recall(expected_ids, actual_ids),
            "ap": self.average_precision(expected_ids, actual_ids),
            "ndcg": self.ndcg(expected_ids, actual_ids)
        }
        return metrics

In [4]:
expected = [1, 2, 3]
actual = [4, 1, 6, 3, 5, 2]

metric_calculator = MetricCalculator(k=5)
results = metric_calculator.evaluate(expected, actual)

for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

hit_rate: 1.0000
mrr: 0.5000
precision: 0.4000
recall: 0.6667
ap: 0.3333
ndcg: 0.4982


In [ ]:
## TODO: Using questions_eval2.jsonl (p0-9999) file as ground truth, Search using milvus and calculate metrics above

In [5]:
### Generate question based on chunks
import openai
from openai import OpenAI

openai_client = OpenAI(api_key = "")

In [6]:
DEFAULT_QA_GENERATE_PROMPT_TMPL = """\
Context information is below.

---------------------
{context_str}
---------------------

Based on the context provided and without relying on prior knowledge, please generate questions related to the given information.

You are a Teacher/Professor preparing for an upcoming quiz or examination. Your task is to create {num_questions_per_chunk} diverse questions that cover different aspects of the document. Ensure the questions are clearly based on the context provided. Write the questions in Vietnamese.
"""

In [7]:
data_folder_path = '../backend/test_data'
df = pd.read_parquet(f"{data_folder_path}/p0-9999/encoded_data.parquet")

In [8]:
len(df)

10012

In [9]:
from bs4 import BeautifulSoup

def clean_html(html_text):
    """
    Function to remove HTML tags from a string and return plain text.
    :param html_text: String containing HTML content.
    :return: Cleaned text without HTML tags.
    """
    # Parse the HTML content
    soup = BeautifulSoup(html_text, "html.parser")
    
    # Get text with all HTML tags removed
    cleaned_text = soup.get_text(separator=' ')
    
    # Return the cleaned text
    return cleaned_text.strip()

In [10]:
def get_prompt(content):
    
    clean_text = clean_html(content)
    
    generated_prompt = DEFAULT_QA_GENERATE_PROMPT_TMPL.format(
        context_str=clean_text, num_questions_per_chunk=2
    )
    
    return generated_prompt

In [11]:
def call_gpt(generated_prompt):
    model_response = openai_client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = 
                        [
                            {"role": "system", "content": ""},
                            {"role": "user", "content": generated_prompt}
                        ]
        )
    response_content = model_response.choices[0].message.content
    return response_content

In [19]:
import random

random_ids = random.sample(range(1, len(df)), 50)
sum = 0
for id in random_ids:
    content = df['content'][id]
    clean_text = clean_html(content)
    sum += len(clean_text)
    # print(len(content))
print(sum)

1245313


In [20]:
question_output = {}
print(len(random_ids))

50


In [21]:
for id in random_ids:
    try:
        prompt = get_prompt(df['content'][id])
        gpt_response = call_gpt(prompt)
        question_output[df['id'][id]] = gpt_response
    except Exception as e:
        print("failed")
        print(df['id'][id])

In [22]:
question_output

{9350: '1. Theo Quyết định số 883/QĐ-UBND ngày 23 tháng 3 năm 2024, có bao nhiêu quy trình nội bộ giải quyết thủ tục hành chính đã được phê duyệt? Hãy liệt kê một vài quy trình trong số đó và nêu rõ lĩnh vực mà chúng thuộc về.\n\n2. Quyết định nêu rõ các cơ quan có trách nhiệm nào trong việc thực hiện các quy trình nội bộ giải quyết thủ tục hành chính đã được phê duyệt? Nêu cụ thể nghĩa vụ của các cơ quan này.',
 3218: '1. Theo Quyết định số 997/QĐ-UBND, danh mục thủ tục hành chính nào đã được công bố trong lĩnh vực tài nguyên nước thuộc phạm vi chức năng quản lý nhà nước của Sở Tài nguyên và Môi trường vào ngày 20 tháng 6 năm 2024?\n\n2. Quyết định số 997/QĐ-UBND có hiệu lực từ ngày nào và các thủ tục hành chính nào được cho là hết hiệu lực thi hành theo quyết định này?',
 5728: '1. Nhà hát Ca múa nhạc Thăng Long có những chức năng chính nào theo quy định tại Quyết định số 2554/QĐ-UBND?\n\n2. Để thực hiện nhiệm vụ quản lý và phát triển, Nhà hát Ca múa nhạc Thăng Long cần phối hợp với 

In [24]:
tmp_res = {str(key): value for key, value in question_output.items()}
import json
with open('questions.jsonl', 'w') as fp:
    json.dump(tmp_res, fp)

In [26]:
import re
def convert_to_list_by_index(input_string):
    items = re.split(r'\d+\.\s', input_string)
    return [item.strip() for item in items if item.strip()]

In [27]:
with open("questions_eval2.jsonl", 'w') as jsonl_file:
    # Convert each key-value pair into a JSON object and write to a new line
    for key, value in tmp_res.items():
        list_value = convert_to_list_by_index(value)
        json_line = json.dumps({key: list_value})
        jsonl_file.write(json_line + '\n')